<p>
<img src="../imgs/EII-ULPGC-logo.jpeg" width="430px" align="right">

# **NOTEBOOK 7.3**
---

# **Flamingo**

Flamingo (DeepMind, 2022) es uno de los modelos multimodales más influyentes previos a la era de los modelos unificados como GPT‑4o y Gemini, [Alayrac et al., 2022](https://arxiv.org/pdf/2204.14198).

## **¿Cómo funciona Flamingo?**

Flamingo es un modelo multimodal diseñado para procesar y generar texto condicionado por imágenes o secuencias de vídeo. Su arquitectura se basa en la combinación de cuatro componentes principales:

1. **Vision Encoder**: Un modelo de visión previamente entrenado (como un Vision Transformer o una red convolutiva) que convierte las imágenes en representaciones visuales significativas. Este módulo está congelado y no se entrena durante el proceso de Flamingo.

2. **Perceiver Resampler**: Un módulo entrenable que toma la salida del Vision Encoder y la transforma en un conjunto compacto de representaciones latentes de tamaño fijo. Este resampler utiliza atención cruzada para condensar la información visual, permitiendo que el modelo maneje múltiples imágenes o frames de vídeo de manera eficiente.

3. **Language Model (LLM)**: Un modelo de lenguaje grande y previamente entrenado (como Chinchilla) que genera texto autoregresivamente. Este modelo también está congelado y no se entrena durante Flamingo.

4. **Gated Cross-Attention**: Un mecanismo entrenable que se inserta entre las capas del LLM para permitir que el modelo consulte las representaciones latentes producidas por el Perceiver Resampler. Este mecanismo permite al modelo decidir dinámicamente cuánta información visual incorporar en cada paso de generación.

<br>
<br>


<div align="center">
<img src="./imgs/flamingo.png" alt="Flamingo Architecture" width="900"/>
</div>

Veamos ahora cada uno de estos componentes con más detalle.

### **Vision Encoder**

El Vision Encoder es el encargado de transformar cada imagen en una secuencia de representaciones visuales significativas. Flamingo no entrena este módulo: utiliza un modelo de visión previamente entrenado, normalmente un **Vision Transformer (ViT)** o un **convolutional vision backbone** de gran escala. Este encoder produce una secuencia de tokens visuales que capturan tanto la estructura espacial como la semántica de la imagen.

La clave es que el Vision Encoder está **completamente congelado**, es decir, sus parámetros no se actualizan durante el entrenamiento de Flamingo. Esto garantiza dos cosas: primero, que el modelo aprovecha el conocimiento visual robusto ya aprendido; y segundo, que el coste computacional se reduce enormemente, porque Flamingo no tiene que volver a entrenar un modelo visual masivo. De esta manera, Flamingo delega la percepción pura en un módulo visual consolidado y se centra únicamente en aprender cómo integrar esa percepción en un sistema de razonamiento lingüístico.

### **Perceiver Resampler**

El Perceiver Resampler constituye uno de los elementos más característicos de la arquitectura Flamingo. Su función principal es transformar una secuencia visual muy extensa, producida por el encoder de visión, en un conjunto reducido de representaciones latentes de tamaño fijo. Este proceso permite que el modelo de lenguaje, congelado y de naturaleza textual, pueda acceder a la información visual de manera controlada y eficiente durante el razonamiento multimodal.

El punto de partida del Resampler es la salida del encoder visual. Dependiendo de la arquitectura utilizada (una red convolutiva como NFNet o un Vision Transformer) esta salida puede adoptar la forma de un mapa espacial de características o de una larga secuencia de *patch embeddings*. En cualquier caso, el encoder produce una colección de tokens visuales capaces de describir la estructura semántica de la imagen, aunque suelen ser demasiados para ser manejados directamente por el modelo de lenguaje. De hecho, un LLM estándar no está preparado para recibir cientos o miles de tokens visuales sin perder eficiencia o saturarse. La misión del Perceiver Resampler consiste precisamente en condensar esa información en un número mucho más pequeño de tokens que el LLM podrá consultar a lo largo de su proceso de generación.

Cuando Flamingo procesa vídeo, marca cada token visual con un embedding temporal que indica a qué frame pertenece. Este embedding se suma a los embeddings espaciales producidos por el encoder visual. El Perceiver Resampler, mediante sus rondas de cross-attention, aprende a integrar estas señales espacio-temporales para producir latentes que resumen no solo la información espacial, sino también la evolución temporal del vídeo. De esta forma, el LLM recibe una representación compacta que incluye la dinámica temporal sin necesidad de procesar explícitamente secuencias largas de frames.

Para lograr esta compresión inteligente, lo primero que introduce el módulo es un conjunto fijo de vectores, denominados *latentes aprendidos*. Estos vectores no proceden de la imagen, sino que son parámetros del modelo, inicializados de forma aleatoria y optimizados posteriormente durante el entrenamiento multimodal. Puede imaginarse cada latente como una “consulta abstracta” que el modelo ha aprendido a formular: cada uno representa un tipo de información que es útil extraer de la secuencia visual. Durante el entrenamiento, estos latentes van especializándose para capturar diferentes aspectos de la escena: relaciones espaciales, contexto global, propiedades de los objetos, o cualquier rasgo relevante para las tareas posteriores de generación de lenguaje.

El mecanismo mediante el cual estos latentes absorben información visual es la atención cruzada. En cada bloque del Perceiver Resampler, los latentes actúan como *queries*, mientras que los tokens visuales se proyectan a espacios de *keys* y *values* a través de matrices entrenables $W_K$ y $W_V$. Los latentes proyectados generan sus queries mediante una matriz $W_Q$. El resultado es que cada latente "atiende" a los tokens visuales en función de la relevancia que tengan para ese vector. La atención cruzada aplica una operación de la forma

$$
\mathrm{CrossAttn}(Q, K, V) = \mathrm{softmax}\left(\frac{QK^\top}{\sqrt{d}}\right) V,
$$

donde los latentes proyectados sirven como queries y la secuencia visual proyectada como keys y values. Con ello, cada latente recoge información ponderada de toda la imagen, no de una región concreta, sino de cualquier parte de la secuencia visual que tenga relación con el tipo de representación que está aprendiendo a formar.

Tras esta etapa de atención cruzada, cada latente se procesa mediante un pequeño bloque tipo Transformer, formado por normalización y una red feed-forward con conexiones residuales. Este bloque permite refinar la información almacenada por los latentes, estabilizarla y enriquecerla. El proceso de atención cruzada seguido de feed-forward puede repetirse varias veces —normalmente entre dos y seis en Flamingo— lo que da lugar a una secuencia de refinamientos sucesivos. Con cada repetición, los latentes se vuelven representaciones más abstractas, densas y útiles para las tareas multimodales del modelo.

A diferencia de los Perceiver originales, el Resampler de Flamingo suele prescindir del mecanismo de self-attention entre latentes, ya que su propósito no es establecer relaciones internas complejas entre ellos, sino más bien extraer información relevante de los tokens visuales. Esta simplificación hace el módulo más eficiente y evita el coste computacional cuadrático de la atención total entre latentes.

El resultado final del Perceiver Resampler es un conjunto compacto de tokens latentes cuya cantidad permanece fija independientemente de la resolución de la imagen, del número de patches o incluso del número de imágenes o frames que se hayan proporcionado como entrada. Este conjunto de latentes constituye la memoria visual accesible para el modelo de lenguaje. Durante la generación multimodal, el LLM consulta estos latentes a través de las capas de *gated cross-attention*, recuperando la información visual cuando la necesita para producir descripciones, responder preguntas o razonar sobre la escena.

En resumen, el Perceiver Resampler no copia ni reduce la imagen por métodos tradicionales, sino que aprende un mecanismo dinámico para seleccionar y condensar la información visual a través de latentes entrenables y atención cruzada. Esta arquitectura hace posible que Flamingo integre información visual compleja en un modelo de lenguaje congelado, permitiendo un razonamiento multimodal eficaz sin necesidad de reentrenar ni el encoder visual ni el propio LLM.

<div align="center">
<img src="./imgs/perceiver_resampler.png" alt="Perceiver Resampler Architecture" width="700"/>
</div>

Este módulo **sí se entrena desde cero**, a diferencia del Vision Encoder. Su razonamiento es análogo al de un *resumidor visual*: convierte un número arbitrario de entradas visuales en una representación densa y manejable. Esto permite a Flamingo procesar varias imágenes o incluso secuencias enteras de vídeo con mucha eficiencia, manteniendo un tamaño de entrada visual compatible con un LLM.

### **Language Model**

El núcleo de Flamingo es un **modelo de lenguaje de tipo decoder**, un LLM grande y previamente entrenado, como GPT-style Transformers. En el artículo original se usó **Chinchilla**, aunque Flamingo puede integrarse con otros LLMs similares. Este modelo tampoco se entrena durante Flamingo. Sus pesos están **congelados**, de modo que conserva su estructura lingüística, su conocimiento del mundo y su capacidad de razonamiento en contexto.

La función del LLM es generar texto autoregresivamente, pero ahora con la posibilidad de consultar la información visual proporcionada por el Perceiver Resampler. Esto significa que Flamingo reutiliza la enorme potencia lingüística del LLM sin necesidad de volver a entrenarlo, y se limita a ajustar cómo debe incorporar la información visual dentro de esa arquitectura lingüística. El LLM procesa secuencias en las que el texto aparece intercalado con tokens que representan imágenes, y genera una respuesta condicionada tanto por el lenguaje previo como por el contenido visual.


### **Gated Cross-Attention**

Para integrar la información visual dentro del LLM sin alterar su estructura interna, Flamingo introduce un mecanismo específico llamado **Gated Cross-Attention**. Este mecanismo consiste en insertar, entre las capas estándar del LLM, nuevas subcapas de atención cruzada que permiten que el modelo consulte los tokens latentes producidos por el Perceiver Resampler.

La clave de este mecanismo es que está **entrenado desde cero**, mientras que el resto del LLM permanece congelado. La atención cruzada está “gated”, lo que significa que el modelo puede decidir dinámicamente cuánta información visual debe incorporar en cada paso de generación. En algunos fragmentos del texto, la visión puede ser muy relevante (por ejemplo, al describir detalles concretos de una imagen), mientras que en otros el modelo puede ignorarla. El gating proporciona ese control gradual y explícito.

Esta inserción intercalada de cross-attention es justamente la innovación que hace posible que Flamingo sea capaz de **razonar de forma multimodal sin reentrenar el LLM completo**. El lenguaje sigue siendo el motor de la generación, pero ahora puede acceder de manera eficiente y flexible a la memoria visual condensada.

<div align="center">
<img src="./imgs/gated.png" alt="GXAttn Architecture" width="900"/>
</div>

## **Entrenamiento**

El entrenamiento de Flamingo se basa en una estrategia en la que ni el encoder visual ni el modelo de lenguaje se actualizan. Ambos permanecen completamente congelados. El proceso de aprendizaje se centra únicamente en los módulos que conectan visión y lenguaje: el Perceiver Resampler y las capas de gated cross-attention intercaladas dentro del LLM. Durante el entrenamiento, Flamingo recibe secuencias multimodales que combinan imágenes o frames de vídeo con texto asociado. El objetivo es que el modelo aprenda a integrar la representación visual condensada producida por el Resampler en el proceso de generación lingüística del LLM.

Para ello, Flamingo utiliza datasets multimodales a gran escala. DeepMind emplea enormes conjuntos de datos imagen-texto obtenidos de la web, similares a ALIGN o LAION, pero privados y filtrados de manera interna. Además, el modelo se entrena con grandes corpus de vídeo acompañados de texto, como HowTo100M, además de otros conjuntos web-scale de vídeo con subtítulos o descripciones automáticas. También se incluyen datasets públicos como Conceptual Captions, CC12M, Kinetics o Something-Something. Esta combinación de imágenes, vídeos y textos narrativos permite a Flamingo aprender a relacionar información visual con secuencias de lenguaje natural en múltiples contextos.

La función de pérdida utilizada es exactamente la misma que en un modelo de lenguaje autoregresivo: una cross-entropy que se aplica sobre los tokens del texto generado. Dado el texto objetivo, el modelo debe predecir cada token condicionándose en la secuencia previa y en los latentes visuales extraídos del Resampler. Esta pérdida lingüística es la única que se aplica durante todo el entrenamiento. Aunque el LLM está congelado, los gradientes fluyen hacia el Perceiver Resampler y hacia las capas de atención cruzada, que aprenden a extraer información útil de los tokens visuales y a integrarla en el razonamiento lingüístico. El encoder visual, por su parte, nunca recibe gradientes. Gracias a esta estrategia, Flamingo puede adquirir capacidades multimodales profundas sin necesidad de reentrenar ni la percepción visual ni el modelo de lenguaje propiamente dicho.